In [1]:
import numpy as np

In [2]:
import onnxruntime as ort

onnx_model_path = "hair_classifier_v1.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

In [3]:
inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

In [4]:
output_name

'output'

In [5]:
from io import BytesIO
from urllib import request

from PIL import Image

In [6]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [7]:
url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'

In [8]:
input_size = 200

In [9]:
img = download_image(url)

img = prepare_image(img, (input_size,input_size))

In [10]:
X = np.array(img)
X = np.expand_dims(X, axis=0)

In [11]:
def preprocess_pytorch_style(X):
    # X: shape (1, 299, 299, 3), dtype=float32, values in [0, 255]
    X = X / 255.0

    mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
    std = np.array([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1)

    # Convert NHWC → NCHW
    # from (batch, height, width, channels) → (batch, channels, height, width)
    X = X.transpose(0, 3, 1, 2)  

    # Normalize
    X = (X - mean) / std

    return X.astype(np.float32)

In [12]:
input_array = preprocess_pytorch_style(X)

In [14]:
tensor[:, 0, 0]

tensor([-1.0733, -0.2150, -1.4210])

In [13]:
input_array[0]

array([[[[-1.0732939 , -1.004795  , -1.0390444 , ..., -1.0732939 ,
          -1.0732939 , -1.210292  ],
         [-1.0561692 , -1.0219197 , -1.004795  , ..., -1.0219197 ,
          -1.0561692 , -1.1760424 ],
         [-0.9534207 , -0.9705454 , -0.91917115, ..., -1.0219197 ,
          -1.1075435 , -1.2274168 ],
         ...,
         [-1.6726604 , -1.6726604 , -1.5185375 , ...,  1.7351657 ,
           1.6495419 ,  1.7865399 ],
         [-1.6726604 , -1.6555356 , -1.6384108 , ...,  1.6837914 ,
           1.5981677 ,  1.7009162 ],
         [-1.6384108 , -1.7240345 , -1.6726604 , ...,  1.718041  ,
           1.7351657 ,  1.6837914 ]],

        [[-0.214986  , -0.10994398, -0.10994398, ..., -0.5126051 ,
          -0.47759104, -0.63515407],
         [-0.197479  , -0.16246499, -0.16246499, ..., -0.5126051 ,
          -0.54761904, -0.60014004],
         [-0.07492997, -0.12745099, -0.14495799, ..., -0.47759104,
          -0.60014004, -0.7051821 ],
         ...,
         [-1.1428572 , -1.1428572 

In [15]:
tensor = tensor.unsqueeze(0)

In [16]:
input_array = tensor.numpy().astype(np.float32)

In [17]:
results = session.run([output_name], {input_name: input_array})

In [19]:
predictions = results[0][0].tolist()

In [20]:
predictions

[0.09156641364097595]